In [146]:
inputs = ["""
Agent: Thank you for calling [Hospital Name]. My name is Emily. How can I assist you today?

User: Hi Emily, I'm calling to complain about a bad experience I had during my recent visit to your hospital.

Agent: I'm sorry to hear that you had a negative experience. I'm here to listen and assist you. Could you please provide me with your full name and the date of your visit?

User: My name is Mark Johnson, and I visited your hospital on June 15th.

Agent: Thank you, Mr. Johnson. I apologize for any inconvenience caused. Could you please share some details about the specific issue or incident that occurred during your visit?

User: Well, I had to wait for over two hours in the waiting area before I was called in for my appointment. It was frustrating and uncomfortable.

Agent: I understand how frustrating that can be, Mr. Johnson. I apologize for the long wait time. We strive to provide timely and efficient service to all our patients. I will make a note of your complaint and forward it to the appropriate department for review. Is there anything else you would like to share about your experience?

User: Yes, during my appointment, the doctor seemed rushed and didn't take enough time to listen to my concerns. I felt like my questions were brushed off.

Agent: I'm sorry to hear that, Mr. Johnson. It's important for our doctors to provide attentive and compassionate care. I will make sure to pass along your feedback to the doctor's supervisor for further investigation. We appreciate you bringing this to our attention. Is there anything else you would like to add?

User: No, that's all. I just wanted to express my dissatisfaction with the service I received.

Agent: I appreciate you taking the time to share your concerns, Mr. Johnson. Your feedback helps us improve our services. Rest assured, we will take the necessary steps to address the issues you've raised. If you have any further questions or if there's anything else we can do to assist you, please don't hesitate to reach out. Thank you for bringing this to our attention.

User: Thank you, Emily. I hope my feedback helps in improving the quality of service at your hospital.

Agent: You're welcome, Mr. Johnson. We value your feedback and will work towards providing a better experience for all our patients. If you have any further concerns in the future, please don't hesitate to contact us. Have a great day!

User: Thank you, Emily. Goodbye.

Agent: Goodbye, Mr. Johnson. Take care.
""", """
Agent: Thank you for calling [Clinic Name]. My name is Lisa. How can I assist you today?

User: Hi Lisa, I wanted to inquire if your clinic provides physical therapy services?

Agent: Absolutely, we do offer physical therapy services. May I ask if you are looking for a specific type of physical therapy?

User: Yes, I'm specifically looking for aquatic therapy. Do you have that available?

Agent: Yes, we do offer aquatic therapy at our clinic. It's a great option for patients who benefit from the buoyancy and resistance provided by water. Would you like me to provide you with more information about our aquatic therapy program?

User: Yes, please. I would like to know more about the availability and the process to schedule an appointment.

Agent: Certainly. Our aquatic therapy sessions are available on weekdays from 9 am to 5 pm. To schedule an appointment, we would need some basic information from you. May I have your full name and contact number, please?

User: Sure, my name is Sarah Johnson and my contact number is 555-6789.

Agent: Thank you, Ms. Johnson. Now, before we proceed, I would like to inform you that a referral from your primary care physician may be required for aquatic therapy. Have you already obtained a referral?

User: No, I haven't obtained a referral yet. Can you guide me on how to get one?

Agent: Of course, Ms. Johnson. To obtain a referral, you will need to contact your primary care physician's office. They will evaluate your condition and determine if aquatic therapy is appropriate for you. If you need assistance finding their contact information, I can help you with that as well.

User: That would be helpful. I'm not sure if I have their number handy.

Agent: Not a problem, Ms. Johnson. Let me quickly look up the contact information for your primary care physician. Can you please provide me with their name?

User: Yes, my primary care physician's name is Dr. Michael Anderson.

Agent: Thank you. I have found the contact information for Dr. Michael Anderson's office. Their phone number is 555-1234. I recommend giving them a call to discuss your need for a referral for aquatic therapy.

User: Thank you, Lisa. I'll contact Dr. Anderson's office right away.

Agent: You're welcome, Ms. Johnson. Once you have obtained the referral, please give us a call back, and we will be more than happy to assist you in scheduling your aquatic therapy sessions. Is there anything else I can assist you with today?

User: No, that's all for now. Thank you for your help, Lisa.

Agent: You're welcome, Ms. Johnson. I hope everything goes smoothly with obtaining the referral. If you have any further questions or need any additional assistance, please don't hesitate to reach out. Have a great day!

User: Thank you, Lisa. You too. Goodbye.

Agent: Goodbye, Ms. Johnson. Take care.
""", """
Agent: Thank you for calling our hospital's appointment line. How may I assist you today?

User: Hi, I need to schedule an appointment for a specialized surgery.

Agent: I'd be happy to help you with that. Before we proceed, may I have your name, please?

User: Yes, my name is John Smith.

Agent: Thank you, Mr. Smith. Could you please provide me with your date of birth?

User: Sure, it's January 15th, 1980.

Agent: Great, thank you. Now, could you please let me know the reason for your appointment and the name of the specialist you would like to see?

User: I need to schedule an appointment for a knee replacement surgery with Dr. Johnson.

Agent: I see. In order to proceed with scheduling the appointment, we require a referral from your primary care physician. Do you have a referral for this surgery?

User: Oh, I wasn't aware that I needed a referral. I haven't obtained one yet.

Agent: No problem, Mr. Smith. It's important to have a referral to ensure that you receive the appropriate care. I recommend contacting your primary care physician to obtain the referral. Once you have it, please call us back, and we'll be more than happy to assist you in scheduling the appointment.

User: Alright, I understand. I'll reach out to my doctor and get the referral as soon as possible. Can you provide me with the contact information to make it easier?

Agent: Absolutely, Mr. Smith. Our hospital's main line is 555-1234. Once you have the referral, please call this number, and our appointment line will be ready to assist you further.

User: Thank you for your help. I'll contact my doctor and then call back to schedule the appointment.

Agent: You're welcome, Mr. Smith. We appreciate your understanding. If you have any further questions or need assistance, please don't hesitate to reach out. We look forward to hearing from you soon. Have a great day!

User: Thank you, you too. Goodbye.

Agent: Goodbye, Mr. Smith. Take care.
""","""
Agent: Thank you for calling [Hospital Name]. My name is Lisa. How can I assist you today?

User: Finally, someone answered! I've been trying to reach you for ages. I need to change my scheduled appointment, and I'm getting really frustrated with your hospital's lack of response.

Agent: I apologize for the inconvenience and the delay in reaching us, sir. I understand your frustration, and I'm here to assist you with changing your appointment. May I have your full name and the date of your current appointment, please?

User: My name is James Thompson, and my appointment is scheduled for next Monday, the 25th.

Agent: Thank you, Mr. Thompson. I apologize for any inconvenience caused by the delay in getting back to you. Let me check the availability and see if we can reschedule your appointment. Could you please let me know your preferred date and time for the new appointment?

User: I don't care about the date and time. I just need it to be as soon as possible. I've been waiting for this appointment for weeks, and now I have to change it. It's ridiculous!

Agent: I understand your urgency, Mr. Thompson, and I apologize for any inconvenience caused. Let me check the availability for the earliest possible appointment. Please bear with me for a moment.

User: Fine, but make it quick. I don't have all day.

Agent: Thank you for your patience, Mr. Thompson. I have found an available slot for you tomorrow, the 23rd, at 10 am. Would that work for you?

User: Tomorrow? That's too soon! I have other commitments. Can't you find anything later in the week?

Agent: I apologize for the inconvenience, Mr. Thompson. Let me check for other available options later in the week. Please hold on for a moment.

User: Hurry up, please. I don't have time to waste.

Agent: Thank you for waiting, Mr. Thompson. I have found an available slot for you on Friday, the 29th, at 2 pm. Would that be more convenient for you?

User: Finally, that works. But I'm still frustrated with the lack of response and the inconvenience caused. This whole process has been a nightmare.

Agent: I apologize once again for the inconvenience and frustration caused, Mr. Thompson. Your feedback is important to us, and I will make sure to pass it along to the appropriate department for review. Is there anything else I can assist you with?

User: No, that's all. I just hope the rescheduled appointment goes smoothly.

Agent: I understand your concern, Mr. Thompson. Rest assured, I will make a note of your request and ensure that your rescheduled appointment is properly updated in our system. If you have any further questions or need any additional assistance, please don't hesitate to reach out. Thank you for your patience.

User: Whatever. Just make sure it's done. Goodbye.

Agent: Goodbye, Mr. Thompson. Take care.
""","""
Agent: Thank you for calling [Hospital Name]. My name is Sarah. How may I assist you today?

User: Hi Sarah, my name is Lisa, and I'm calling from XYZ Insurance. I have a patient who needs to schedule a surgery at your hospital. I would like to verify their insurance coverage and discuss the billing process.

Agent: Thank you for reaching out, Lisa. I'm here to assist you with the insurance verification and billing process. Could you please provide me with the patient's full name and date of birth?

User: Sure, the patient's name is John Smith, and his date of birth is January 15th, 1980.

Agent: Thank you, Lisa. Let me check our system to verify the insurance coverage for Mr. Smith. Please bear with me for a moment.

User: Take your time, Sarah. I appreciate your help.

Agent: Thank you for your patience, Lisa. I have verified Mr. Smith's insurance coverage. He is covered under our network with XYZ Insurance. However, I would like to confirm if there are any specific requirements or pre-authorization needed for the surgery.

User: That's great to hear, Sarah. As for the pre-authorization, I have already obtained it from our insurance company. The surgery has been approved, and we have the necessary documentation.

Agent: Wonderful, Lisa. That will make the process smoother. Now, regarding the billing process, we will submit the claim directly to XYZ Insurance on behalf of Mr. Smith. Is there any specific information or forms that you require from our end?

User: Thank you, Sarah. We appreciate your assistance. As for the billing process, we would need an itemized bill for the surgery, along with any supporting medical records or documentation related to the procedure. This will help us process the claim efficiently.

Agent: Noted, Lisa. We will ensure that an itemized bill, along with the necessary medical records, is provided to you promptly. If you have any further questions or need any additional information, please don't hesitate to reach out.

User: Thank you, Sarah. I appreciate your help and prompt response. If there are any updates or changes regarding the surgery or billing, I will be in touch.

Agent: You're welcome, Lisa. We're here to assist you. Feel free to contact us anytime if you need any further assistance or have any updates. Have a great day!

User: Thank you, Sarah. You too. Goodbye.

Agent: Goodbye, Lisa. Take care.
""","""
Agent: Thank you for calling [Hospital Name]. My name is Emily. How can I assist you today?

User: Hi Emily, I'm calling to make an appointment for my child with Dr. Johnson. We've been referred to him, and we really need to see him as soon as possible.

Agent: I understand your urgency, ma'am. Dr. Johnson is a highly sought-after doctor, and his schedule tends to fill up quickly. Let me check his availability for you. May I have your child's full name and date of birth, please?

User: Yes, my child's name is Emma Thompson, and her date of birth is June 10th, 2010.

Agent: Thank you, Mrs. Thompson. Let me check Dr. Johnson's schedule for the next available appointment. Please bear with me for a moment.

User: Thank you, Emily. I appreciate your help.

Agent: I apologize for the inconvenience, Mrs. Thompson, but it seems that Dr. Johnson's schedule is currently full for the next two weeks. However, we do have other highly qualified doctors in the same specialty who may be able to see your child sooner. Would you like me to check their availability for you?

User: I really prefer Dr. Johnson because he was highly recommended by our pediatrician. Is there any way we can get on a waiting list or be notified if there are any cancellations?

Agent: I completely understand your preference, Mrs. Thompson. We do have a waiting list for Dr. Johnson's appointments. I can add your child's name to the list, and if there are any cancellations or earlier availability, we will contact you immediately. Would you like me to proceed with adding Emma's name to the waiting list?

User: Yes, please. That would be great. We really hope to see Dr. Johnson soon.

Agent: Noted, Mrs. Thompson. I have added Emma's name to the waiting list for Dr. Johnson's appointments. We will do our best to accommodate your request and notify you as soon as there is an opening. In the meantime, I can also help you schedule an appointment with one of our other experienced doctors in the same specialty. Would you like me to assist you with that?

User: Yes, I guess that would be the best option for now. Please go ahead and schedule an appointment with another doctor.

Agent: Thank you for understanding, Mrs. Thompson. Let me check the availability of our other doctors in the same specialty. Please hold on for a moment.

User: Alright, I'll wait. Thank you for your help, Emily.

Agent: Thank you for your patience, Mrs. Thompson. I have found an available appointment with Dr. Anderson, another highly qualified doctor in the same specialty. The earliest appointment I have is next Tuesday, the 30th, at 2 pm. Would that work for you?

User: Yes, that works for us. Please go ahead and schedule the appointment with Dr. Anderson.

Agent: Noted, Mrs. Thompson. I have successfully scheduled an appointment for Emma with Dr. Anderson on Tuesday, the 30th, at 2 pm. You will receive a confirmation email shortly with all the details. If there are any changes or updates, please don't hesitate to contact us. Is there anything else I can assist you with?

User: No, that's all for now. Thank you for your help, Emily.

Agent: You're welcome, Mrs. Thompson. I'm glad I could assist you. If you have any further questions or need any additional assistance, please don't hesitate to reach out. Have a great day!

User: Thank you, Emily. You too. Goodbye.

Agent: Goodbye, Mrs. Thompson. Take care.
""", """
Agent: Thank you for calling [Hospital Name]. My name is Sarah. How may I assist you today?

User: Hi Sarah, I wanted to schedule an appointment for a chiropractic treatment.

Agent: I apologize for any inconvenience, but we do not offer chiropractic services at our hospital. However, we have a range of other medical services available. Is there anything else I can assist you with?

User: Are you kidding me? This is ridiculous! I specifically called because I need chiropractic treatment. Why don't you offer it?

Agent: I understand your frustration, and I apologize for any inconvenience caused. Our hospital focuses on providing specialized medical services, and chiropractic treatment is not currently part of our offerings. I apologize if this was not made clear to you earlier. Is there anything else I can assist you with?

User: No, I don't need anything else. This is just a waste of my time. Can I at least know who I'm speaking with?

Agent: I apologize for not introducing myself earlier. My name is Sarah. I understand your frustration, and I'm sorry that we couldn't meet your specific needs. If you have any further questions or if there's anything else I can assist you with, please don't hesitate to reach out. Thank you for calling.

User: Whatever. Goodbye.

Agent: Goodbye. Take care.
"""]

In [144]:
from dotenv import load_dotenv
load_dotenv()  # take environment variables from .env.
import gradio as gr
import openai
import re
import json

def bad_math(num1, num2):
    result = float(num1) + 1.2 + float(num2)
    return str(result)

# ---------------------------------------------------------------------------------------
def needs_tool(response):
    return "Tool:" in response

def extract_call(string):
    # regex pattern
    pattern = r'Tool: (\w+)\((.*?)\)'
    match = re.search(pattern, string)
    if match:
        tool_name = match.group(1)
        parameters = match.group(2).replace('"', '').split(', ')
        return tool_name, parameters
    else:
        return None, None
    
def invoke_tool(response):
    tool_name, parameters = extract_call(response)
    
    if tool_name == "bad_math":
        tool_result = bad_math(*parameters)
        
    return tool_result

def is_valid_json(json_str):
    try:
        json.loads(json_str)
        return True
    except ValueError:
        return False
    
def create_csv_file(file_path, data):
    # Check if the CSV file already exists
    file_exists = os.path.isfile(file_path)

    # Open the CSV file in append mode
    with open(file_path, 'a', newline='') as csv_file:
        writer = csv.DictWriter(csv_file, fieldnames=data.keys())

        # Write the header row if the file is newly created
        if not file_exists:
            writer.writeheader()

        # Write the data row
        writer.writerow(data)

# ---------------------------------------------------------------------------------------
# Define a function to get the AI's reply using the OpenAI API
def get_ai_reply(message, model="gpt-3.5-turbo-16k", system_message=None, temperature=0, message_history=[]):
    # Initialize the messages list
    messages = []
    
    # Add the system message to the messages list
    if system_message is not None:
        messages += [{"role": "system", "content": system_message}]

    # Add the message history to the messages list
    if message_history is not None:
        messages += message_history
    
    if message is not None:
        # Add the user's message to the messages list
        messages += [{"role": "user", "content": message}]
    
    # Make an API call to the OpenAI ChatCompletion endpoint with the model and messages
    completion = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature
    )
    
    # Extract and return the AI's response from the API response
    return completion.choices[0].message.content.strip()
# ---------------------------------------------------------------------------------------

In [141]:
# Define a function to handle the chat interaction with the AI model
def chat(message, chatbot_messages, history_state):
    # Initialize chatbot_messages and history_state if they are not provided
    chatbot_messages = chatbot_messages or []
    history_state = history_state or []
    csv_file_path = 'data.csv'
    
    # Try to get the AI's reply using the get_ai_reply function
    try:
        prompt = """
        You are an AI assistant for a medical call center. The center's function is to take, record call information, and assist callers.
                
        ##Rules
        Don't exlcude any information, even if N/A.
        
        ##Analyze the transcribed audio and gather the following information:
        - name_of_patient
        - name_of_caller (patient or person on behalf of the patient.)
        - identity_of_caller [patient, patient_care_giver, insurance_representative, other_medical_office, other]
        - reason_for_calling [make_appointment, insurance_verification, change_appointment, cancel_appointment, request_test_results, request_perscription_refill, request_referral, inquiry_about_services, request_medical_records, provide_feedback, other]
        - appointment_scheduled [true, false, N/A] (True if appointment is successfully scheduled for a date, false if appointment cancelled or failed to schdule, N/A if not appilicable)
        - reason_for_schdule_failure [N/A, no_availability, no_referral, not_a_service, insurance_converage, incomplete_inforamtion, other]
        - problems_raised_by_caller (if no problem: N/A)
        - sentiment_of_call [positive, negative, neutral]
        """
        prompt2 = """
        You are a helpful JSON creator.

        Take the input and output it into JSON using the following keys and types:
        - name_of_patient [string]
        - name_of_caller [string]
        - identity_of_caller [string]
        - reason_for_calling [string]
        - appointment_scheduled [bool or null]
        - reason_for_schedule_failure [string or null]
        - problems_raised [string or null]
        - sentiment_of_call [string]
        """
        
        ai_reply = get_ai_reply(message, model="gpt-3.5-turbo-16k", system_message=prompt.strip(), message_history=history_state)
            
        # Append the user's message and the AI's reply to the history_state list
        history_state.append({"role": "user", "content": message})
        history_state.append({"role": "assistant", "content": ai_reply})
        
        ai_reply = get_ai_reply(ai_reply, model="gpt-3.5-turbo-16k", system_message=prompt2.strip(), message_history=history_state)
        history_state.append({"role": "assistant", "content": ai_reply})
        
#         ##Add entry to CSV
#         if(is_valid_json(ai_reply)):
#             json_data = json.loads(ai_reply)
#             create_csv_file(csv_file_path, json_data)
        
        # Append the user's message and the AI's reply to the chatbot_messages list for the UI
        chatbot_messages.append((message, ai_reply))

        # Return None (empty out the user's message textbox), the updated chatbot_messages, and the updated history_state
    except Exception as e:
        # If an error occurs, raise a Gradio error
        raise gr.Error(e)
        
    return None, chatbot_messages, history_state

# Define a function to launch the chatbot interface using Gradio
def get_chatbot_app():
    # Create the Gradio interface using the Blocks layout
    with gr.Blocks() as app:
        # Create a chatbot interface for the conversation
        chatbot = gr.Chatbot(label="Conversation")
        # Create a textbox for the user's message
        message = gr.Textbox(label="Message")
        # Create a state object to store the conversation history
        history_state = gr.State()
        # Create a button to send the user's message
        btn = gr.Button(value="Send")

        # Connect the send button to the chat function
        btn.click(chat, inputs=[message, chatbot, history_state], outputs=[message, chatbot, history_state])
        # Return the app
        return app
# ---------------------------------------------------------------------------------------        
# Call the launch_chatbot function to start the chatbot interface using Gradio
app = get_chatbot_app()
app.queue()  # this is to be able to queue multiple requests at once
app.launch(share=True)

Running on local URL:  http://127.0.0.1:7955
Running on public URL: https://e9320365fc1d5e6cce.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [145]:
prompt = """
You are an AI assistant for a medical call center. The center's function is to take, record call information, and assist callers.

##Rules
Don't exlcude any information, even if N/A.

##Analyze the transcribed audio and gather the following information:
- name_of_patient
- name_of_caller (patient or person on behalf of the patient.)
- identity_of_caller [patient, patient_care_giver, insurance_representative, other_medical_office, other]
- reason_for_calling [make_appointment, insurance_verification, change_appointment, cancel_appointment, request_test_results, request_perscription_refill, request_referral, inquiry_about_services, request_medical_records, provide_feedback, other]
- appointment_scheduled [true, false, N/A] (True if appointment is successfully scheduled for a date, false if appointment cancelled or failed to schdule, N/A if not appilicable)
- reason_for_schdule_failure [N/A, no_availability, no_referral, not_a_service, insurance_converage, incomplete_inforamtion, other]
- problems_raised_by_caller (if no problem: N/A)
- sentiment_of_call [positive, negative, neutral]
"""
prompt2 = """
You are a helpful JSON creator.

Take the input and output it into JSON using the following keys and types:
- name_of_patient [string]
- name_of_caller [string]
- identity_of_caller [string]
- reason_for_calling [string]
- appointment_scheduled [bool or null]
- reason_for_schedule_failure [string or null]
- problems_raised [string or null]
- sentiment_of_call [string]
"""

for i in range(len(inputs)):
    history_state = []
    csv_file_path = 'data.csv'
    
    ai_reply = get_ai_reply(inputs[i], model="gpt-3.5-turbo-16k", system_message=prompt.strip(), message_history=history_state)

    # Append the user's message and the AI's reply to the history_state list
    history_state.append({"role": "user", "content": inputs[i]})
    history_state.append({"role": "assistant", "content": ai_reply})

    ai_reply = get_ai_reply(ai_reply, model="gpt-3.5-turbo-16k", system_message=prompt2.strip(), message_history=history_state)
    history_state.append({"role": "assistant", "content": ai_reply})

    ##Add entry to CSV
    if(is_valid_json(ai_reply)):
        json_data = json.loads(ai_reply)
        create_csv_file(csv_file_path, json_data)

print("done")

done
